In [1]:
!pip install skl2onnx onnx onnxmltools

  Using cached skl2onnx-1.7.0-py2.py3-none-any.whl (191 kB)
  Using cached onnxmltools-1.7.0-py2.py3-none-any.whl (252 kB)
  Using cached onnxconverter_common-1.7.0-py2.py3-none-any.whl (64 kB)
  Using cached keras2onnx-1.7.0-py3-none-any.whl (96 kB)
  Using cached fire-0.3.1.tar.gz (81 kB)
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=771d43d8451dfcbda6c1066b6e619b3f1ddec020d8ecc3fb0d8c0e70bbf8e526
  Stored in directory: /home/bentoml/.cache/pip/wheels/95/38/e1/8b62337a8ecf5728bdc1017e828f253f7a9cf25db999861bec
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=4551679660493ab6f79c2ab0add35ed6f744f524e43039ab340d2abc7d796707
  Stored in directory: /home/bentoml/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built fire termcolor


In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

from sklearn.linear_model import LogisticRegression
clr = LogisticRegression()
clr.fit(X_train, y_train)

/opt/anaconda3/envs/bentoml-dev-py36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
clr.predict([[5.1, 3.5, 1.4, 0.2]])

array([0])

In [20]:
%%writefile iris_classifier_onnx.py

import numpy
import bentoml
from bentoml.adapters import DataframeInput
from bentoml.artifact import OnnxModelArtifact

@bentoml.artifacts([OnnxModelArtifact('model')])
@bentoml.env(auto_pip_dependencies=True)
class IrisClassifierOnnx(bentoml.BentoService):
    
    @bentoml.api(input=DataframeInput())
    def predict(self, df):
        input_data = df.to_numpy().astype(numpy.float32)
        input_name = self.artifacts.model.get_inputs()[0].name
        output_name = self.artifacts.model.get_outputs()[0].name
        outputs = numpy.zeros(input_data.shape[0])
        for i in range(input_data.shape[0]):
            outputs[i] = self.artifacts.model.run([output_name], {input_name: input_data[i: i + 1]})[0]
        return outputs

Overwriting iris_classifier_onnx.py


In [7]:
from iris_classifier_onnx import IrisClassifierOnnx
from skl2onnx.common.data_types import FloatTensorType
import onnxmltools



initial_type = [('float_input', FloatTensorType([None, 4]))]
onnx_model = onnxmltools.convert_sklearn(clr, initial_types=initial_type)

svc = IrisClassifierOnnx()
svc.pack('model', onnx_model)

saved_path = svc.save()

[2020-08-04 09:59:26,083] INFO - Detect BentoML installed in development model, copying local BentoML module file to target saved bundle path
running sdist
running egg_info
writing BentoML.egg-info/PKG-INFO
writing dependency_links to BentoML.egg-info/dependency_links.txt
writing entry points to BentoML.egg-info/entry_points.txt
writing requirements to BentoML.egg-info/requires.txt
writing top-level names to BentoML.egg-info/top_level.txt
reading manifest file 'BentoML.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'


no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.8.3+47.g5daa71b
creating BentoML-0.8.3+47.g5daa71b/BentoML.egg-info
creating BentoML-0.8.3+47.g5daa71b/bentoml
creating BentoML-0.8.3+47.g5daa71b/bentoml/adapters
creating BentoML-0.8.3+47.g5daa71b/bentoml/artifact
creating BentoML-0.8.3+47.g5daa71b/bentoml/cli
creating BentoML-0.8.3+47.g5daa71b/bentoml/clipper
creating BentoML-0.8.3+47.g5daa71b/bentoml/configuration
creating BentoML-0.8.3+47.g5daa71b/bentoml/configuration/__pycache__
creating BentoML-0.8.3+47.g5daa71b/bentoml/handlers
creating BentoML-0.8.3+47.g5daa71b/bentoml/marshal
creating BentoML-0.8.3+47.g5daa71b/bentoml/saved_bundle
creating BentoML-0.8.3+47.g5daa71b/bentoml/server
creating BentoML-0.8.3+47.g5daa71b/bentoml/utils
creating BentoML-0.8.3+47.g5daa71b/bentoml/yatai
creating BentoML-0.8.3+47.g5daa71b/bentoml/yatai/client
creating BentoML-0.8.3+47.g5daa71b/bentoml/yatai/deployment
creating BentoML-0.8.3+47.g5daa71b/bentoml/yatai/dep

### Use BentoService with BentoML CLI


Use bentoml get to list all versions of BentoService, including the version tag will display additional information and metadata

In [15]:
!bentoml get IrisClassifierOnnx

BENTO_SERVICE                             AGE                         APIS                                   ARTIFACTS
IrisClassifierOnnx:20200804094745_743CE1  2.36 seconds                predict<DataframeInput:DefaultOutput>  model<OnnxModelArtifact>
IrisClassifierOnnx:20200804094714_1C3FB7  33.12 seconds               predict<DataframeInput:DefaultOutput>  model<OnnxModelArtifact>
IrisClassifierOnnx:20200804094615_182CB5  1 minute and 32.66 seconds  predict<DataframeInput:DefaultOutput>  model<OnnxModelArtifact>
IrisClassifierOnnx:20200803180418_647884  15 hours and 43 minutes     predict<DataframeInput:DefaultOutput>  model<OnnxModelArtifact>


In [8]:
!bentoml get IrisClassifierOnnx:latest

[2020-05-17 11:46:53,198] INFO - Getting latest version IrisClassifierOnnx:20200517114625_DB7077
{
  "name": "IrisClassifierOnnx",
  "version": "20200517114625_DB7077",
  "uri": {
    "type": "LOCAL",
    "uri": "/Users/bozhaoyu/bentoml/repository/IrisClassifierOnnx/20200517114625_DB7077"
  },
  "bentoServiceMetadata": {
    "name": "IrisClassifierOnnx",
    "version": "20200517114625_DB7077",
    "createdAt": "2020-05-17T18:46:41.959251Z",
    "env": {
      "condaEnv": "name: bentoml-IrisClassifierOnnx\nchannels:\n- defaults\ndependencies:\n- python=3.7.3\n- pip\n",
      "pipDependencies": "onnxruntime\nbentoml==0.7.5\nnumpy==1.18.4",
      "pythonVersion": "3.7.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "OnnxModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "DataframeHandler",
        "docs": "BentoService API",
        "handlerConfig": {
          "orient": "records",
          "typ": "f

In [23]:
!bentoml run IrisClassifierOnnx:latest predict --input '[[5.1, 3.5, 1.4, 0.2], [5.1, 3.5, 1.4, 0.2]]'

[2020-08-04 09:52:10,335] INFO - Getting latest version IrisClassifierOnnx:20200804094903_8746D7
[2020-08-04 09:52:11,204] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-08-04 09:52:11,238] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.1, but loading from BentoML version 0.8.1+27.g8e155f5.dirty
[2020-08-04 09:52:12,134] INFO - Initializing onnxruntime InferenceSession from onnx file:'/home/bentoml/bentoml/repository/IrisClassifierOnnx/20200804094903_8746D7/IrisClassifierOnnx/artifacts/model.onnx'
[0. 0.]


### Model Serving via REST API


Note: Running as local rest api server does not work with Google Colab, please copy this notebook to run it locally

In [2]:
!bentoml serve IrisClassifierOnnx:latest --enable-microbatch

[2020-08-04 09:58:22,230] INFO - Getting latest version IrisClassifierOnnx:20200804094903_8746D7
[2020-08-04 09:58:22,230] INFO - Starting BentoML API server in development mode..
[2020-08-04 09:58:23,377] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-08-04 09:58:23,405] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.1, but loading from BentoML version 0.8.3+47.g5daa71b
[2020-08-04 09:58:23,406] WARNING - Saved BentoService Python version mismatch: loading BentoService bundle created with Python version 3.7.4, but current environment version is 3.6.10.
[2020-08-04 09:58:24,464] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.1, but loading from BentoML version 0.8.

#### Send prediction requeset to the REST API server

```
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '[[5.1, 3.5, 1.4, 0.2]]'
```



### Install saved BentoService as PyPI package


In [8]:
!pip install --quiet {saved_path}

In [13]:
!IrisClassifierOnnx info

[2020-05-17 11:49:26,394] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.5. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-17 11:49:26,410] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.5,  but loading from BentoML version 0.7.5+21.g2b3d2c8.dirty
{
  "name": "IrisClassifierOnnx",
  "version": "20200517114625_DB7077",
  "created_at": "2020-05-17T18:46:41.959251Z",
  "env": {
    "conda_env": "name: bentoml-IrisClassifierOnnx\nchannels:\n- defaults\ndependencies:\n-

In [9]:
!IrisClassifierOnnx run predict --input '[[5.1, 3.5, 1.4, 0.2]]'

[0.]


### Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For this project, we are going to deploy to AWS lambda

`bentoml lambda deploy` command deploys to AWS lambda with a single command

In [24]:
!bentoml lambda deploy onnx-iris -b IrisClassifierOnnx:{svc.version} --api-name predict --debug

[2020-05-17 12:08:36,339] DEBUG - Creating local YataiService instance
[2020-05-17 12:08:36,544] DEBUG - Upgrading tables to the latest revision
Deploying "IrisClassifierOnnx:20200517114625_DB7077" to AWS Lambda |[2020-05-17 12:08:37,645] DEBUG - Creating s3 bucket: btml-dev-onnx-iris-4d3d6d in region us-west-2
\[2020-05-17 12:08:38,536] DEBUG - Created temporary directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-xszwplmj
[2020-05-17 12:08:38,536] DEBUG - Generating cloudformation template.yaml for lambda project at /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-xszwplmj
[2020-05-17 12:08:38,546] DEBUG - Validating generated template.yaml
[2020-05-17 12:08:38,546] DEBUG - Setting envar "AWS_DEFAULT_REGION" to us-west-2 for subprocess call
-[2020-05-17 12:08:41,120] DEBUG - SAM cmd ['sam', 'validate', '--template-file', '/private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-xszwplmj/template.yaml', '--region', 'us-we

In [25]:
!curl -X POST \
  https://kfov6vj4kf.execute-api.us-west-2.amazonaws.com/Prod/predict \
  -H 'Content-Type: application/json' \
  -d '[[5.1, 3.5, 1.4, 0.2]]'

[0]

In [26]:
!bentoml lambda delete onnx-iris

Successfully deleted AWS Lambda deployment "onnx-iris"
